# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [9]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Assembly bias and galaxy conformity in cosmo-dc2	Talk	talk-1	"III Workshop on the halo-galaxy
Connection"	2023-07-01	Atacama, Chile	https://sites.google.com/uda.cl/atacama-2023/	In my talk I will present my partial results in the search for Galaxy Assembly Bias in observational data. We focus on how the alignments between central and satellite galaxies vary according to their morphology and color (known as galactic conformity), using cosmoDC2 galaxy catalogs. These catalogs were designed to generate a data-set similar to future LSST surveys to obtain predictions that can be evaluated in the context of future LSST results.
Evolution of the dark matter haloes shapes	Talk	talk-2	"4th Regional Extragalactic Astronomy
Meeting, 60 Years Of The Sersic Law"	2023-12-04	Córdoba, Argentina	https://extragalacticmeeting.conicet.unc.edu.ar/	In this work, we aim at investigating the morphology evolution of Milky Way mass-like dark matter ha

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Assembly bias and galaxy conformity in cosmo-dc2,Talk,talk-1,III Workshop on the halo-galaxy\nConnection,2023-07-01,"Atacama, Chile",https://sites.google.com/uda.cl/atacama-2023/,In my talk I will present my partial results i...
1,Evolution of the dark matter haloes shapes,Talk,talk-2,"4th Regional Extragalactic Astronomy\nMeeting,...",2023-12-04,"Córdoba, Argentina",https://extragalacticmeeting.conicet.unc.edu.ar/,"In this work, we aim at investigating the morp..."
2,Fingerprints of modified gravity effects on ga...,Talk,talk-3,1 st Rio DM meeting,2023-05-01,"Rio de Janeiro,Brazil",https://www.youtube.com/@CBPFMCTI,We search for detectable signatures of 𝑓 (𝑅) g...
3,Redshift evolution of the dark matter haloes s...,Talk,talk-4,A journey through galactic environments,2023-10-01,"Porto Ercole, Italy",https://galaxylifecycle.wixsite.com/galacticjo...,"In this work, we aim at investigating the morp..."
4,Feedback por binarias de rayos x sobre el ism ...,Poster,Poster-1,64th Annual Reunion of the Argentinian Astrono...,2022-09-01,"Buenos Aires,Argentina",https://www.fisica.org.ar/2022/05/05/64a-reuni...,Las caracterçisticas de las primeras generacio...
5,Study on the evolution of dark matter haloes i...,Talk,Talk-5,64th Annual Reunion of the Argentinian Astrono...,2022-09-01,"Buenos Aires,Argentina",https://www.fisica.org.ar/2022/05/05/64a-reuni...,This article analyze the dark matter (dm) halo...
6,Fingerprints of modified gravity effects on g...,Talk,Talk-6,63th Annual Reunion of the Argentinian Astrono...,2021-09-01,"San Luis, Argentina",https://www.fisica.org.ar/2022/05/05/63a-reuni...,We found in this article that for a given stel...
7,Baryons shaping dark matter haloes,Talk,Talk-7,II Reunion Bi-National AAA-SOCHIAS,2018-10-01,"La Serena, Chile",https://sochias.cl/second-binational-meeting-a...,In this work we aim at investigating the effec...
8,Studying the dark matter halo shape,Talk,Talk-8,Friends-of-friends virtual meeting,2018-08-01,"Cordoba, Argentina",NaN,NaN
9,Estudio numérico de los halos de materia\noscu...,Talk,Talk-9,n 60th Annual Reunion of the Argentinian Astro...,2017-08-01,"Mendoza, Argentina",http://www.astronomiaargentina.org.ar/reuniones,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [13]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [14]:
!ls ../_talks

2017-08-01-Talk-9.md  2022-09-01-Poster-1.md  2023-10-01-talk-4.md
2018-08-01-Talk-8.md  2022-09-01-Talk-5.md    2023-12-04-talk-2.md
2018-10-01-Talk-7.md  2023-05-01-talk-3.md
2021-09-01-Talk-6.md  2023-07-01-talk-1.md


In [15]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
